# rasterio 101: part 1 

The `rasterio` is a Python standard for the raster data library that expresses GDAL's data using fewer non-idiomatic extension classes and more idiomatic Python types and protocols while maintaining GDAL's Python binding speed. 

For this study, we will be using a clipped Sentinel-2 image with 11 bands which started at band 2. So, following the Sentinel-2 image band sequences respectively, it starts with band 2; Blue band, band 3; Green band and so forth to the 11th band. 

References:
- [rasterio: Python Quickstart]('https://rasterio.readthedocs.io/en/latest/quickstart.html')
- [AutoGIS: Reading raster files with Rasterio]('https://autogis-site.readthedocs.io/en/latest/notebooks/Raster/reading-raster.html')
- [Geo-Python 2021]('https://geo-python-site.readthedocs.io/en/latest/')
- [Pyton Basics: List Comprehensions, Dictionary Comprehensions and Generator Expressions]('https://www.netguru.com/blog/python-list-comprehension-dictionary')
- [Numpy Tutorials: Linear algebra on n-dimensional arrays]('https://numpy.org/numpy-tutorials/content/tutorial-svd.html')
- [W3 Schools: Python Dictionaries]('https://www.w3schools.com/python/python_dictionaries.asp')
- [The Python Tutorial: 3. An Informal Introduction to Python]('https://docs.python.org/3/tutorial/introduction.html#first-steps-towards-programming')
- [StackOverflow: What is the meaning of curly braces?]('https://stackoverflow.com/questions/9197324/what-is-the-meaning-of-curly-braces#:~:text=%22Curly%20Braces%22%20are%20used%20in,%22%20%3A%20%22Banana%22%2C%20%7D')

In [1]:
import os
from osgeo import gdal
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rio.__version__

'1.2.9'

In [3]:
os.getcwd()

'D:\\deeplearning\\script\\jupyter'

## Opening raster dataset in reading mode

### `rasterio.open()`

The `rasterio.open()`takes the data path and returns it as an opened dataset object. Dataset objects have some of the same attributes as Python file objects. 

In [4]:
dataset = rio.open(r'D:\deeplearning\data\subset_s2_01_corrected_02_resampled_03.tif')

### `.mode`

The dataset `.mode` function will confirm the type of file with abbreviations. The mode `r` refers to the data in 'read' form

In [5]:
dataset.mode

'r'

### `.closed`

The dataset `.closed` function is to test if the dataset is closed. It will return 'False' should the data is opened.

In [6]:
dataset.closed

False

## Raster dataset attributes

### `.count` 

Image dataset objects have bands and we can check the number of bands by using `.count` function

In [7]:
dataset.count

11

### `.width` and `.height`

The dataset object width and height can be checked using the `.width` and `.height` functions.

In [8]:
dataset.width
dataset.height
print(dataset.width, dataset.height)

3432 2173


### dictionary comprehension `.indexes` and `.dtypes`

Some dataset attributes returns the attribute properties of all dataset bands through tuple of values; one for each band. To get a mapping of band indexes to variable data types, apply a **dictionary comprehension** to the `zip()` product of the dataset's `indexes` and `dtypes` attribute.
   - _dictionary_: a collection which is ordered (as of Python version 3.7), changeable but does not allow duplicated, used to store data values in **key:value** pairs. See [Python Dictionaries]('https://www.w3schools.com/python/python_dictionaries.asp')
   - _list & dictionary comprehension_: see [Python Basics: List Comprehensions, Dictionary Comprehensions and Generator Expressions]('https://www.netguru.com/blog/python-list-comprehension-dictionary')

In [9]:
{i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}

{1: 'uint16',
 2: 'uint16',
 3: 'uint16',
 4: 'uint16',
 5: 'uint16',
 6: 'uint16',
 7: 'uint16',
 8: 'uint16',
 9: 'uint16',
 10: 'uint16',
 11: 'uint16'}

### `.driver`

The `.driver` attribute reveals the type of raster format of the raster dataset

In [38]:
dataset.driver

'GTiff'

## Raster data georeferences

### `.bounds`

GIS raster dataset is different from typical image file since its pixels or in this Python data context, 'elements' are mapped to regions on the earth surface. Thus, all of these elements are contained within a bounding box.

For the raster utilized in this exercise, the return of the `.bounds` function is the values making up the bounding box of the raster; _left, bottom, right, top_. It can be understood as:
> - The dataset covers the world from 748410 meters to 782730 meters, left to right. And it covers 333540 meters to 355270 meters, bottoms up.
> - Overall, it covers 34320 meters width and  21730 meters height

In [10]:
dataset.bounds

BoundingBox(left=748410.0, bottom=333540.0, right=782730.0, top=355270.0)

### `.transform`

The `.bounds` attribute originated from `.transform` attribute which is the affine matrix that maps pixel locations in coordinates to spatial locations; (rows and columns to x and y). 
> - The origin of this matrix (0, 0) is the upper left corner of the dataset; thus so is the spatial position.

In [11]:
dataset.transform

Affine(10.0, 0.0, 748410.0,
       0.0, -10.0, 355270.0)

In [12]:
# We can find the position of the origin (0, 0); upper left corner of the data
dataset.transform * (0, 0)

(748410.0, 355270.0)

In [13]:
# And similarly the extremeties/total extent end; lower right corner
dataset.transform * (dataset.width, dataset.height)

(782730.0, 333540.0)

### `.crs`

The special attribute to geospatial raster coordinate values are relative to the dataset's coordinate reference system (CRS). These coordinate reference system are in the EPSG format.

In [14]:
dataset.crs

CRS.from_epsg(32647)

## Reading the raster data

### `.indexes`

The bands of a raster image that has been opened can be **accessed** through the bands' index number. 
- It is to be noted that, bands indexed in the raster image starts from 1
- This is a convetion inherited from GDAL

In [15]:
# This will give full list number of bands contained in the opened image
dataset.indexes

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)

### `.read()`

For the opened raster dataset to be used in any operations, it should be read into Python. That can be achieved by implementing the `.read()` method. 
- The `.read()` method will return a `numpy` N-D array
- Any operations that is doable for numpay arrays can be implemented on the arrays that have been read from the raster image dataset

In [29]:
# We can read out all the bands in the single image
raster_array = dataset.read()

# Or we can call them in singles
raster_array_b2 = dataset.read(1)

# And both will still be an array
print(type(raster_array), type(raster_array_b2))

# And we can still be look at the shape of both
print(raster_array.shape, raster_array_b2.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(11, 2173, 3432) (2173, 3432)


In [30]:
# View the elements of raster_array
raster_array

array([[[ 710,  744,  694, ...,  348,  344,  367],
        [ 703,  786,  691, ...,  357,  357,  365],
        [ 728,  738,  672, ...,  360,  357,  351],
        ...,
        [ 252,  277,  271, ...,  703,  664,  650],
        [ 244,  253,  253, ...,  724,  696,  646],
        [ 252,  265,  250, ...,  732,  659,  658]],

       [[ 856,  835,  889, ...,  543,  569,  581],
        [ 894,  899,  867, ...,  558,  593,  561],
        [ 871,  907,  816, ...,  546,  586,  533],
        ...,
        [ 478,  479,  505, ...,  778,  808,  753],
        [ 440,  472,  457, ...,  834,  752,  704],
        [ 460,  457,  465, ...,  976,  827,  778]],

       [[ 431,  421,  448, ...,  349,  354,  360],
        [ 451,  481,  459, ...,  373,  368,  360],
        [ 430,  489,  436, ...,  371,  391,  368],
        ...,
        [ 221,  229,  229, ...,  893,  999,  758],
        [ 217,  229,  228, ...,  901,  873,  726],
        [ 227,  232,  225, ..., 1070,  927,  795]],

       ...,

       [[ 536,  536,  53

In [31]:
# View the elements of raster_array_b2
raster_array_b2

array([[710, 744, 694, ..., 348, 344, 367],
       [703, 786, 691, ..., 357, 357, 365],
       [728, 738, 672, ..., 360, 357, 351],
       ...,
       [252, 277, 271, ..., 703, 664, 650],
       [244, 253, 253, ..., 724, 696, 646],
       [252, 265, 250, ..., 732, 659, 658]], dtype=uint16)

### Band statistics

Givent that we can read all the bands in the single image into a 3D-array, we can also calculate the statistics of each band directly using the attributes and the for-loop.
> - This will iterate through the data withing the 3D-array, extract the information on the attributes requested and compile them in a single list.
> - An empty list needs to be created for this purpose; as often when working with for-loop

The attributes that we are interest in is the statistics of **each single band** which are:
- `.min()`
- `.mean()`
- `.median()`
- `.max()`

**NOTE:** Curly brackets are used to define dictionaries in Python

In [33]:
# The dataset and its bands have already been read as 'raster_array'

# Create the empty list to contain the dictionary of information for each bands
## We are creating a 'key:value' type of data, so dictionary is the best way to list the statistics of each bands
band_stats = []

for band in raster_array:
    band_stats.append({
        'min': band.min(),
        'median': np.median(band),
        'max': band.max(),
        'mean': band.mean()
    })

# Show the statistics
band_stats

[{'min': 90, 'median': 717.0, 'max': 16624, 'mean': 763.1122406317413},
 {'min': 79, 'median': 879.0, 'max': 17520, 'mean': 952.8905765771274},
 {'min': 1, 'median': 562.0, 'max': 16704, 'mean': 823.1337790718255},
 {'min': 1, 'median': 901.0, 'max': 15946, 'mean': 1086.367904549048},
 {'min': 154, 'median': 2039.0, 'max': 15766, 'mean': 1765.5463074852744},
 {'min': 173, 'median': 2382.0, 'max': 16150, 'mean': 2101.8954705556753},
 {'min': 141, 'median': 2308.0, 'max': 15352, 'mean': 2037.1858612318806},
 {'min': 139, 'median': 2529.0, 'max': 15946, 'mean': 2207.8530901335203},
 {'min': 417, 'median': 2534.0, 'max': 5735, 'mean': 2290.7969063265314},
 {'min': 99, 'median': 1780.0, 'max': 14344, 'mean': 1648.1553597499294},
 {'min': 42, 'median': 909.0, 'max': 15149, 'mean': 1188.019502701624}]

## Spatial indexing values in arrays

### `.index()`

The `.index()` method is different from `.indexes` attribute. The `.index()` method extracts the array indices that corresponds to the points in the georeferenced space. It is a roundabout way of getting the specific coordinates using the raster image and calling the index on the raster arrays that has been read.

In [37]:
# Obtain the value for pixel 10 km east and 5 km south of dataset upper left corner (origin)

## 1 Extract information on the rows and columns aka x and y coordinates
x, y = (dataset.bounds.left + 10000, dataset.bounds.top - 5000)
row, col = dataset.index(x, y)

## 2 Obtain the values at the coordinates extracted; row, col
raster_array_b2[row, col]

### Let's look at coordinates and pixel extracted
print(row, col, raster_array_b2[row, col])

500 1000 355


### `.xy()`

The `.xy()` method is used to obtain the value of the pixel at the specific coordinates.

In [34]:
# If we try to obtain the coordinates of the very center of the image
dataset.xy(dataset.height // 2, dataset.width // 2)

(765575.0, 344405.0)

## Opening raster dataset in writing mode

### `.open()`

To save an image n-d array its georeferencing information into a new raster data, we have to call for the method to create a new raster file. This can be done using the similar method as opening the raster dataset in reading mode. This time, we need to specify that we are going to open it in an editable 'writing' mode which is denoted by the abbreviation `w`.

This method also requires a few arguments to generate the new raster dataset:
- _driver_: The name of the desired format
- _width_: The number of columns in the dataset
- _height_: The number of rows in the dataset
- _count_: The count of dataset bands
- _dtype_: The data type of the dataset
- _crs_: The coordinate system in EPSG format to which you want to write the raster as
- _transform_: An affine transformation matrix 
- _nodata_: A 'nodata' value

**NOTE:**
> The _driver, width, height, count_ and _dtype_ keyword arguments parameters is a fixed format-specific properties of the data file 
> Meanwhile _coordinate reference system, transform_ and _nodata value_ configuration are optional

An example can be seen below where we attempt to create a new raster 'new.tif' with parameters similar to an existing raster called 'raster_array' 3D-array:

In [51]:
# Creating new raster
new_dataset = rio.open('new.tif',
                      'w',
                      driver = 'GTiff',
                      height = raster_array_b2.shape[0],
                      width = raster_array_b2.shape[1],
                      count = 1,
                      dtype = raster_array_b2.dtype,
                      crs = 'proj=latlong',  # CRS of an equirectangular coordinate reference system with unit decimal degress
                      transform=dataset.transform)

CPLE_AppDefinedError: Deleting new.tif failed: Permission denied

## Saving the raster dataset

### `.write()`

Call the new dataset `.write()` method to copy the grid into the newly opened dataset with the grid and target band number as arguments.

In [49]:
new_dataset.write(raster_array_b2, 1)

### `.close()`

After writing the grid and information of the new dataset into the newly created 'new.tif' raster, we need to call upon the `.close()` method to sync the data to the disk and finish

In [52]:
new_dataset.close()

In [53]:
# To simplify the process above and include the writing of the new raster, you ca implement context manager protocol

with rio.open('new.tif',
             'w', 
              driver = 'GTiff',
              height = raster_array_b2.shape[0],
              width = raster_array_b2.shape[1],
              count = 1,
              dtype = raster_array_b2.dtype,
              crs = 'proj=latlong',  # CRS of an equirectangular coordinate reference system with unit decimal degress
              transform=dataset.transform
) as dst:
    dst.write(raster_array_b2, 1)